# Install library

In [1]:
# # Install package for inferences
# !pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl
# !pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl
# !pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl
!pip install -qq git+https://github.com/huggingface/transformers.git  -U 
!pip install -qq git+https://github.com/huggingface/accelerate.git  -U 
!pip install -qq git+https://github.com/huggingface/optimum.git  -U 
!pip install -qq bitsandbytes 
!pip install -qq git+https://github.com/huggingface/peft.git  -U 

# !pip install -q /kaggle/input/autocorrect/autocorrect-2.6.1.tar
# # Eanble 4-bit CUDA functions for PyTorch
# #!pip install -q bitsandbytes --no-index --find-link /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl
# #!pip install -q accelerate --no-index --find-link /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl 

In [2]:
# Install packaages for training on TPUs (notebook internet must enable)
!pip install -qq sentencepiece==0.1.99 
!pip install -qq torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q # Updating torch since we need the latest version
!pip install -qq torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q
!pip uninstall -qq tensorflow -y # If we don't do this, TF will take over TPU and cause permission error for PT
# !cp /kaggle/input/utils-xla/spmd_util.py . # From this repo: https://github.com/HeegyuKim/torch-xla-SPMD

## Imports for training on TPUs

In [3]:
## Imports for TPU XLA 
import torch_xla.debug.profiler as xp
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp # We also import mp modules if we wanna use that for some reason
import torch_xla.distributed.parallel_loader as pl
import torch_xla.test.test_utils as test_utils
import torch_xla.experimental.xla_sharding as xs
import torch_xla.runtime as xr
xr.use_spmd() # To enable PyTorch/XLA SPMD execution mode for automatic parallelization

# "experimental" XLA packages
import torch_xla.experimental.xla_sharding as xs 
from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
from torch_xla.experimental.xla_sharding import Mesh
# from spmd_util import partition_module

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Imports for inference and training

In [4]:
import torch, transformers, sklearn, os, gc, re, random, time, sys
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import (
    AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig,
    LlamaConfig, GemmaConfig, AutoModel, AutoModelForCausalLM,
    DataCollatorWithPadding, AutoConfig, AutoModelForSequenceClassification
) 

from accelerate import cpu_offload, dispatch_model
from accelerate.utils.modeling import infer_auto_device_map
from optimum.bettertransformer import BetterTransformer

from tqdm.auto import tqdm
tqdm.pandas()

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 99

print(f'Torch Version: {torch.__version__}')

Torch Version: 2.1.0+cu121


## Common functions

In [5]:
N_FOLD = 5
SEED = 42
DEBUG = False  # True: trained LLM on small samples for debugging

In [6]:
# Seed the same seed to all 
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(SEED)

# Load training data

In [7]:
# Cross validation
def cv_split(train_data):
    skf = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)
    X = train_data.loc[:, train_data.columns != "label"]
    y = train_data.loc[:, train_data.columns == "label"]
    # Split the train into 5 folds
    for fold, (train_index, valid_index) in enumerate(skf.split(X, X['source'])):
        train_data.loc[valid_index, "fold"] = fold

    print(train_data.groupby("fold")["label"].value_counts())
    return train_data

def load_train_data():
    train_df = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv", sep=',')
    train_prompts_df = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv", sep=',')

    # rename column generated to label and remove used 'id' and 'prompt_id' columns
    # Label: 1 indicates generated texts (by LLMs) 
    train_df['source'] = "train_essay"
    train_df = train_df.rename(columns={'generated': 'label'})
    train_df = train_df.reset_index(drop=True)
    train_df = train_df.drop(['id', 'prompt_id'], axis=1)
    
    # Include external data
    external_df = pd.read_csv("/kaggle/input/daigt-v4-train-dataset/train_v4_drcat_01.csv", sep=',')
    # RDizzl3_seven = True
    excluded_prompt_name_list = ['Distance learning','Grades for extracurricular activities','Summer projects']
    external_df = external_df[~(external_df['prompt_name'].isin(excluded_prompt_name_list))]
    # We only need 'text' and 'label' columns
    external_df = external_df[["text", "label", "source"]]
    
    # Merge train and external data into train_data
    train_data = pd.concat([train_df, external_df])
    train_data.reset_index(inplace=True, drop=True)
    
    # print(f"Train data has shape: {train_data.shape}")
    print(f"Train data {train_data.value_counts('label')}") # 1: generated texts 0: human texts
    return train_data

In [8]:
train_data = load_train_data()
train_data = cv_split(train_data)

# Unicode decode 'NFKD'
train_data['text'] = train_data['text'].str.normalize('NFKD')
train_data['text'] = train_data['text'].str.encode('ascii', errors='ignore').str.decode('utf-8')

# remove trailling white spaces, and ;-$ []()
train_data['text'] = train_data['text'].str.strip()
train_data['text'] = train_data['text'].str.replace(r'[ \t]+$', "", regex=True)
train_data['text'] = train_data['text'].str.replace(r'[\;\-\$]', "", regex=True)
train_data['text'] = train_data['text'].str.replace(r'[\[\]\(\)]', "", regex=True)

AUX_LABEL_MAP = dict(zip(train_data['source'].unique(),range(train_data['source'].nunique())))
train_data['aux_label'] = train_data['source'].map(AUX_LABEL_MAP)

display(train_data.head())

Train data label
1    39491
0    23212
Name: count, dtype: int64
fold  label
0.0   1        7900
      0        4641
1.0   1        7897
      0        4644
2.0   1        7899
      0        4642
3.0   1        7898
      0        4642
4.0   1        7897
      0        4643
Name: count, dtype: int64


,text,label,source,fold,aux_label
0,"Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and ...",0,train_essay,2.0,0
1,"Transportation is a large necessity in most countries worldwide. With no doubt, cars, buses, an...",0,train_essay,2.0,0
2,"""America's love affair with it's vehicles seems to be cooling"" says Elisabeth rosenthal. To und...",0,train_essay,4.0,0
3,How often do you ride in a car? Do you drive a one or any other motor vehicle to work? The stor...,0,train_essay,4.0,0
4,Cars are a wonderful thing. They are perhaps one of the worlds greatest advancements and techno...,0,train_essay,2.0,0


# Obfuscation

In [9]:
# from autocorrect import Speller

# speller = Speller(lang='en', fast=True)
# train_data["text"] = train_data["text"].progress_apply(speller)

# display(train_data.head())

In [10]:
# # https://www3.nd.edu/~busiforc/handouts/cryptography/Letter%20Frequencies.html
# BIGRAMS = ['th', 'he', 'in', 'en', 'nt', 're', 'er', 'an', 'ti', 'es', 'on', 'at', 'se',
#            'nd', 'or', 'ar', 'al', 'te', 'co', 'de', 'to', 'ra', 'et', 'ed']
# TRIGRAMS = ['the', 'and', 'tha', 'ent', 'ing', 'ion', 'tio', 'for','nde', 'has', 'nce']

# def replace_uni(text):
#     bi = random.choice(BIGRAMS[:10])
#     c1,c2 = random.sample(bi, 2)
#     return text.replace(c1, c2)

# def replace_bi(text):
#     bi = random.choice(BIGRAMS)
#     c = random.choice(bi)
#     return text.replace(c, bi)

# def replace_tri(text):
#     ti = random.choice(TRIGRAMS)
#     c = random.choice(ti)
#     return text.replace(c, ti)
    
# def Obfuscate(text, P=0.3):
#     if random.random() < P:
#         n = random.random()
#         if n<0.4:
#             text = replace_uni(text)
#         elif n<0.8:
#             text = replace_bi(text)
#         else:
#             text = replace_tri(text)
#     return text

In [11]:
# train_data.loc[train_data['source']!='train_essay', 'text'] = train_data.loc[train_data['source']!='train_essay', 'text'].progress_apply(Obfuscate)
train_data["text"] = "Check whether the following essay is human written or AI generated? Remember, the text might have undergone multiple stages of obfuscation.\n" + train_data["text"]

display(train_data.head())

,text,label,source,fold,aux_label
0,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,2.0,0
1,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,2.0,0
2,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,4.0,0
3,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,4.0,0
4,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,2.0,0


In [12]:
# from autocorrect import Speller
# from collections import Counter

# def cnt_diffs(text1, text2):
#     cnt1 = Counter(text1)
#     cnt2 = Counter(text2)
#     c1 = (cnt1 - cnt2).most_common(1)
#     c2 = (cnt2 - cnt1).most_common(1)
#     c1 = c1[0][1] if len(c1) else 0
#     c2 = c2[0][1] if len(c2) else 0
#     return max(c1, c2)

# speller = Speller(lang='en', fast=True)
# speller_precise = Speller(lang='en', fast=False)
# train_data["text_spell"] = train_data["text"].progress_apply(speller)

# DIFF_TH = 30
# txt_diff = train_data.apply(lambda r: cnt_diffs(r.text, r.text_spell), axis=1)
# train_data.loc[txt_diff >= DIFF_TH, 'text'] = train_data.loc[txt_diff >= DIFF_TH, 'text'].progress_apply(speller_precise)

# display(train_data.head())
# display(train_data.info())

# Train the LLM model on TPUs

NLTK package is used to correct 

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed
# from textblob import TextBlob

def pre_processing_text(text):
    corrected_text = text.replace('\n', ' ')
    #corrected_text = TextBlob(corrected_text).correct()
    return corrected_text

# def parallelize_processing_texts(texts):
#     corrected_texts = []
#     # Run the spelling correction in parallel with a progress bar
# #     with tqdm(total=len(texts)) as pbar:
#     with ThreadPoolExecutor() as executor:
#         futures = list(executor.map(pre_processing_text, texts))
#         for future in as_completed(futures):
#             corrected_text = future.result()
#             corrected_texts.append(corrected_text)
#         if len(corrected_texts) % 100 == 0:    
#             print(f"Complete {len(corrected_texts)} / {len(texts)}")
#     return corrected_texts

In [14]:
# ====================================================
# MODEL
# ====================================================
class GemmaForSequenceClassification(torch.nn.Module):
    def __init__(self, tokenizer, TARGET_MODEL, DEVICE, model_name='gemma7b'):
        super().__init__()
        self.DEVICE = DEVICE
        base_model = AutoModel.from_pretrained(TARGET_MODEL, torch_dtype=torch.bfloat16, trust_remote_code=True)
        # No idea why this is needed
        base_model.config.pretraining_tp = 1 # 1 is 7b
        # Assign Padding TOKEN
        base_model.config.pad_token_id = tokenizer.pad_token_id
        # LoRa
        peft_config = LoraConfig(
            r=16,  # Use larger 'r' value increase more parameters during training
            lora_alpha=16,
            lora_dropout=0.10,
            bias='none',
            inference_mode=False,
            # Only Use Output and Values Projection
            target_modules=[
                'o_proj',
                'v_proj',
            ],
        )
        # Load the new PEFT model
        self.base_model = get_peft_model(base_model, peft_config)
        # Classification Dropout
        self.head_drop = torch.nn.Dropout(p=0.15)
        # Classification Heads
        self.llm_head = torch.nn.Linear(base_model.config.hidden_size, 1, bias=False).to(self.DEVICE)
        self.llm_head_aux = torch.nn.Linear(base_model.config.hidden_size, len(AUX_LABEL_MAP), bias=False).to(self.DEVICE)
        # Print Model Prameters
        self.base_model.print_trainable_parameters()
        
    def get_seq_lens(self, input_ids):
        return (torch.eq(input_ids, self.base_model.config.pad_token_id).int().argmax(-1) - 1).to(self.DEVICE)
    
    def get_pooled_out(self, hidden_states, input_ids):
        bs_r = torch.arange(hidden_states.shape[0], device=self.DEVICE)
        return hidden_states[bs_r, self.get_seq_lens(input_ids)].to(self.DEVICE)

    def forward(self, input_ids, attention_mask):
        hidden_states = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        outputs = self.get_pooled_out(hidden_states[0].to(dtype=torch.float32), input_ids=input_ids)
        outputs = self.head_drop(outputs)
        return self.llm_head(outputs), self.llm_head_aux(outputs)

# Partition Stratagy

In [15]:
# From this repo: https://github.com/HeegyuKim/torch-xla-SPMD
GEMMA_RULES = (
    ("model\\.embed_tokens", ("mp", "fsdp")),
    ("self_attn\\.(q_proj|k_proj|v_proj)", ("fsdp", "mp")),
    ("self_attn\\.o_proj", ("mp", "fsdp")),
    ("mlp\\.gate_proj", ("fsdp", "mp")),
    ("mlp\\.down_proj", ("mp", "fsdp")),
    ("mlp\\.up_proj", ("fsdp", "mp")),
    ("lm_head", ("fsdp", "mp")),
)

ALL_RULES = [
#     (GPTNeoXConfig, GPTNEOX_RULES),
#     (T5Config, T5_RULES),
#     (LlamaConfig, LLAMA_RULES),
#     (MixtralConfig, MIXTRAL_RULES),
    (GemmaConfig, GEMMA_RULES)
]
def find_rule(model):
    for config, rule in ALL_RULES:
        if model.config.__class__ == config:
            return rule
    raise Exception("unsupported model to partitioning")

In [16]:
strkey2id = {
    "dp": 0,
    "fsdp": 1,
    "mp": 2
}

def partition_module(model, mesh, device=xm.xla_device(), verbose=False):
    partition_specs = find_rule(model)
    rule = [(k, tuple([strkey2id[x] for x in v])) for k, v in partition_specs]
    # print(rule)

    for name, module in model.named_modules():
        module.to(device)
        # print(name, module.__class__.__name__)
        if isinstance(module, (nn.Embedding, nn.Linear)):
            for rule_pattern, spec in rule:
                if re.findall(rule_pattern, name):
                    if verbose:
                        print("match", rule_pattern, name)
                    
                    xs.mark_sharding(module.weight, mesh, spec)
                    break

In [17]:
import ctypes
libc = ctypes.CDLL("libc.so.6")


class TrainModelTPU():
    def __init__(self, fold, epochs, train_data):
        self.fold = fold
        self.train_data = train_data
        self.DEBUG = DEBUG
        self.SEED = SEED
        self.NUM_LABELS = 1 # Total Number of Labels (0:human texts, 1:LLM generated texts)
        self.MAX_LENGTH = 1024
        self.BATCH_SIZE = 16
        self.LR = 3e-4 # Learning rate
        self.WD = 0.1 # Weight Decay Ratio
        self.AUX_WEIGHT = 0.4 # Auxilary Loss Weight
        self.DEVICE = xm.xla_device() # Initialize TPU Device
        self.NUM_EPOCHS = epochs # Number Of Training Epochs
        self.NUM_WARMUP_STEPS = 60 # Number of Warmup Steps
        # Load the baseline model
        self.TARGET_MODEL = "/kaggle/input/gemma/transformers/7b/2"
        # Model saving path
        self.SAVE_PATH = f'/kaggle/working/gemma7b_fold{fold}_TPU'
        self.load_model()  # Load the pretrained LLM and tokenizer 
    
    # Disply trainable layers of LLM
    def display_model_layers(self):        
        # Dispaly trainable layers for verification
        trainable_layers = []
        n_trainable_params = 0
        for name, param in self.model.named_parameters():
            # Layer Parameter Count
            n_params = int(torch.prod(torch.tensor(param.shape)))
            # Only Trainable Layers
            if param.requires_grad:
                # Add Layer Information
                trainable_layers.append({
                    '#param': n_params,
                    'name': name,
                    'dtype': param.data.dtype,
                    'params': param
                })
                n_trainable_params += n_params

        display(pd.DataFrame(trainable_layers))
        print(f"Number of trainable parameters: {n_trainable_params:,} "
              f"Number of trainable layers: {len(trainable_layers)}")
        
    def unfreeze_last_nlayers(self, model, n=1):
        num_layers = model.config.num_hidden_layers
        unfreeze_layers = [f'layers.{i}' for i in range(num_layers-n, num_layers)]
        for name, param in model.named_parameters():
            for u in unfreeze_layers:
                if u in name:
                    param.requires_grad = True
        
    # Load pretrained LLM and tokenizer
    def load_model(self):
        # Load the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.TARGET_MODEL, use_fast=False)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.model = GemmaForSequenceClassification(self.tokenizer, self.TARGET_MODEL, self.DEVICE)
        self.unfreeze_last_nlayers(self.model.base_model, n=1)
#         # No idea why this is needed
#         base_model.config.pretraining_tp = 1 # 1 is 7b
#         # Assign Padding TOKEN
#         base_model.config.pad_token_id = self.tokenizer.pad_token_id

#         # LoRa
#         peft_config = LoraConfig(
#             r=16 if self.DEBUG else 64,  # Use larger 'r' value increase more parameters during training
#             lora_alpha=16,
#             lora_dropout=0.10,
#             bias='none',
#             inference_mode=False,
#             # Only Use Output and Values Projection
#             target_modules=[
#                 'o_proj',
#                 'v_proj',
#             ],
#         )
#         # Create LoRa Model
# #         if self.epoch > 1:
# #             print(f"Continue Training the model from {str(self.TARGET_MODEL)}")
# #             # Continue training PEFT model
# #             self.model = PeftModel.from_pretrained(base_model, str(self.TARGET_MODEL))
# #         else:
#         # Load the new PEFT model
#         self.model = get_peft_model(base_model, peft_config)
#         self.head_drop = torch.nn.Dropout(p=0.1)
#         self.llm_head = torch.nn.Linear(base_model.config.hidden_size, 1, bias=False)
#         # Display Trainable Parameters to make sure we load the model successfully
#         self.model.print_trainable_parameters()
# #         if self.DEBUG:
# #             self.display_model_layers()
        print("Complete loading pretrained LLM model")
    
    def create_optimizer_scheduler(self, STEPS_PER_EPOCH, NUM_EPOCHS):        
        # Optimizer (Adam)
        if self.WD:
            optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.LR, weight_decay=self.LR*self.WD)
        else:
            optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LR)

        # Cosine Learning Rate With Warmup
        lr_scheduler = transformers.get_cosine_schedule_with_warmup(
                                    optimizer=optimizer,
                                    num_warmup_steps=self.NUM_WARMUP_STEPS,
                                    num_training_steps=STEPS_PER_EPOCH * NUM_EPOCHS)
        # Set the data type for the optimizer's state (e.g., momentum buffers)
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor) and state[k].dtype is not torch.float32:
                    state[v] = v.to(dtype=torch.float32)
        print("Complete creating optimizer and lr scheduler")
        print("optimizer", optimizer)
        print("lr_scheduler", lr_scheduler)
        return optimizer, lr_scheduler
        
    def partition_mesh(self):
        # Number of TPU Nodes to ensure we can access TPUs and partition the model into mesh
        num_devices = xr.global_runtime_device_count()
        mesh_shape = (1, num_devices, 1)
        device_ids = np.array(range(num_devices))
        mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))
        partition_module(self.model.base_model, mesh)
        return num_devices, mesh
    
    def tokenize(self, text):
        # Tokenize Data
        tokens = self.tokenizer(text, # Texts
                                padding='max_length', # Pad texts to maximum length
                                max_length=self.MAX_LENGTH, # Maximum token length
                                truncation=True, # Truncate texts if they are too long
                                return_tensors='np', # Return Numpy array
                                )
        # Input IDs are the token IDs
        INPUT_IDS = tokens['input_ids'][:,:self.MAX_LENGTH]
        # Attention Masks to Ignore Padding Tokens
        ATTENTION_MASKS = tokens['attention_mask'][:,:self.MAX_LENGTH]
        return INPUT_IDS, ATTENTION_MASKS
        
     # Create a training dataset 
    def create_dataset(self, TEXTS, GENERATED, AUX_LABELS, N_SAMPLES, mesh):
        IDXS = np.arange(N_SAMPLES-(N_SAMPLES%self.BATCH_SIZE))
        while True:
            # Shuffle Indices
            np.random.shuffle(IDXS)
            # Iterate Over All Indices Once
            for idxs in IDXS.reshape(-1, self.BATCH_SIZE):
                # Tokenize
                input_ids, attention_mask = self.tokenize(TEXTS[idxs].tolist())
                # convert to torch tensors
                input_ids = torch.tensor(input_ids).to(self.DEVICE)
                attention_mask = torch.tensor(attention_mask).to(self.DEVICE)
                labels = torch.tensor(GENERATED[idxs]).to(self.DEVICE)
                aux_labels = torch.tensor(AUX_LABELS[idxs]).to(self.DEVICE)
                # Shard Over TPU Nodes
                xs.mark_sharding(input_ids, mesh, (0, 1))
                xs.mark_sharding(attention_mask, mesh, (0, 1))
                xs.mark_sharding(labels, mesh, (0, 1))
                xs.mark_sharding(aux_labels, mesh, (0, 1))
                yield input_ids, attention_mask, labels, aux_labels
    
    # Save the trained model as output files
    def save_model(self):
        self.model = self.model.cpu()# Move model first on CPU before saving weights 
        self.model.base_model.save_pretrained(self.SAVE_PATH) # Save the entire fine-tuned model
        self.tokenizer.save_pretrained(self.SAVE_PATH) # Save tokenizer for inference
        # Save the LLM classification head
        torch.save(self.model.llm_head.cpu().state_dict(), f"{self.SAVE_PATH}/llm_head.pth")     
        # Save all the trainable parameters
        torch.save(dict([(k,v) for k, v in self.model.named_parameters() if v.requires_grad]), f"{self.SAVE_PATH}/model.pth")
        print(f"Save the model and tokenizers to {self.SAVE_PATH}")
    
      # Validate the model
    def valid_model(self):
        num_devices, mesh = self.partition_mesh()
        # print(f'Number_DEVICES: {num_devices}')
        
        # Use the non-fold dataset as valid dataset
        fold_valid_df = self.train_data[self.train_data["fold"] == self.fold]
        if self.DEBUG:
            fold_valid_df = fold_valid_df.sample(n=100, random_state=self.SEED)  # Validate on small samples 
        
        # Compute total samples and number of steps in one epochs
        N_SAMPLES = len(fold_valid_df)
        print(f"Start validating the model with number of sample {N_SAMPLES}")

        # Generated By AI Label of Texts
        TEXTS = fold_valid_df['text'].values
        GENERATED = fold_valid_df['label'].values.reshape(-1,1).astype(np.float32)
        AUX_LABELS = fold_valid_df['aux_label'].values.reshape(-1,1).astype(np.float32)
        # Create a valid dataset 
        VALID_DATASET = self.create_dataset(TEXTS, GENERATED, AUX_LABELS, N_SAMPLES, mesh)

        # Put Model In Eval Modus
        self.model.eval()
        # Loss Function
        LOSS_FN = torch.nn.BCEWithLogitsLoss().to(dtype=torch.float32)
        METRICS = {'loss': [], 
                   'auc': {'y_true': [], 'y_pred': []} }
        STEPS = N_SAMPLES // self.BATCH_SIZE
        for step in tqdm(range(STEPS)):
            # Enable inference mode using `no_grad`
            with torch.no_grad():
                start = time.time()
                # Get Batch
                input_ids, attention_mask, labels, _ = next(VALID_DATASET)
                 # Forward Pass
                outputs, _ = self.model(input_ids=input_ids, attention_mask=attention_mask)
                # Logits Float32
                logits = outputs.to(dtype=torch.float32)
                # Backward Pass
                loss = LOSS_FN(logits, labels)
                # Update Metrics And Progress Bar
                METRICS['loss'].append(float(loss))
                METRICS['auc']['y_true'] += labels.squeeze().tolist()
                METRICS['auc']['y_pred'] += logits.sigmoid().tolist()

        # Compute and display the validation results
        print(f"Number of validation data {len(fold_valid_df)}\n"
              f"loss (valid): {np.mean(METRICS['loss']): .3f}\n"
              f"auc (valid): {sklearn.metrics.roc_auc_score(METRICS['auc']['y_true'], METRICS['auc']['y_pred']):.3f}")
 
    # Train the model by the fold data
    def train_model(self):
        num_devices, mesh = self.partition_mesh()
        print(f'Number_DEVICES: {num_devices}')
        # Use the fold data as training data
        fold_train_df = self.train_data[self.train_data["fold"] != self.fold]
        if self.DEBUG:
            fold_train_df = fold_train_df.sample(frac =.1, random_state=SEED) # Select a small amount of samples 
        # limited to two columns
        fold_train_df = fold_train_df[['text', 'label', 'aux_label']]
#         print("Start correcting the typos in training dataset")
#         start = time.time()
#         # Preprocess the text to correct the typos 
#         fold_train_df['text'] = fold_train_df['text'].map(lambda text: pre_processing_text(text))
#         print(f"Complete correcting the texts {len(fold_train_df['text'])} in {time.time() - start : .1f}")
        # Compute total samples and number of steps in one epochs
        N_SAMPLES = len(fold_train_df)
        # Compute the total steps per epochs
        STEPS_PER_EPOCH = N_SAMPLES // self.BATCH_SIZE
        print(f'BATCH_SIZE: {self.BATCH_SIZE}, N_SAMPLES: {N_SAMPLES}, STEPS_PER_EPOCH: {STEPS_PER_EPOCH}')
        
        # Generated By AI Label of Texts
        TEXTS = fold_train_df['text'].values
        GENERATED = fold_train_df['label'].values.reshape(-1,1).astype(np.float32)
        AUX_LABELS = fold_train_df['aux_label'].values.reshape(-1,1).astype(np.float32)
        print(f'TEXTS shape: {TEXTS.shape}\n'
              f'AUX_LABELS shape: {AUX_LABELS.shape}\n'
              f'GENERATED shape: {GENERATED.shape}')
        
        # Create a train dataset
        TRAIN_DATASET = self.create_dataset(TEXTS, GENERATED, AUX_LABELS, N_SAMPLES, mesh)
        
        # Create optimizer and lr_scheduler
        optimizer, lr_scheduler = self.create_optimizer_scheduler(STEPS_PER_EPOCH, self.NUM_EPOCHS)
        
        # Put Model In Train Modus
        self.model.train()
        # Loss Function, basic Binary Cross Entropy
        LOSS_FN = torch.nn.BCEWithLogitsLoss().to(dtype=torch.float32)
        LOSS_FN_AUX = torch.nn.CrossEntropyLoss(label_smoothing=0.1).to(dtype=torch.float32)
        # Training loop goes through each epoch
        for epoch in tqdm(range(self.NUM_EPOCHS)):
            METRICS = {'loss': [], 
                       'auc': {'y_true': [], 'y_pred': []} }
            # Go through each step
            for step in range(STEPS_PER_EPOCH):
                # Zero Out Gradients
                optimizer.zero_grad()
                # Get Batch
                input_ids, attention_mask, labels, aux_labels = next(TRAIN_DATASET)
                # Test the TRAIN_DATASET for debugging first record
                # Forward Pass
                outputs, aux_outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                # Logits Float32
                logits = outputs.to(dtype=torch.float32)
                aux_logits = aux_outputs.to(dtype=torch.float32)
                # Backward Pass
                if self.AUX_WEIGHT:
                    loss = LOSS_FN(logits, labels)
                    aux_loss = LOSS_FN_AUX(aux_logits, aux_labels.squeeze())
                    loss = (loss * (1-self.AUX_WEIGHT)) + (aux_loss * self.AUX_WEIGHT)
                    loss.backward()
                else:
                    loss = LOSS_FN(logits, labels)
                    loss.backward()
                # Update Weights
                optimizer.step()
                xm.mark_step()
                # Update Learning Rate Scheduler
                lr_scheduler.step()
                # Update Metrics And Progress Bar
                METRICS['loss'].append(float(loss))
                METRICS['auc']['y_true'] += labels.squeeze().tolist()
                METRICS['auc']['y_pred'] += logits.sigmoid().tolist()
                # print(f"Complete updating metrics {METRICS}")
                # Metrics Shown After Both Classes Present
                if np.unique(METRICS['auc']['y_true']).size == 2:
                    metrics = 'µ_loss: {:.3f}'.format(np.mean(METRICS['loss']))
                    metrics += ', step_loss: {:.3f}'.format(METRICS['loss'][-1])
                    metrics += ', µ_auc: {:.3f}'.format(
                        sklearn.metrics.roc_auc_score(METRICS['auc']['y_true'], METRICS['auc']['y_pred'])
                    )

                    lr = optimizer.param_groups[0]['lr']
                    print('\r'*100, f'{epoch+1:02}/{self.NUM_EPOCHS:02} | {step+1:04}/{STEPS_PER_EPOCH} lr: {lr:.2E}, {metrics}', end='')
            avg_loss = np.mean(METRICS['loss'])
            roc_auc_score = sklearn.metrics.roc_auc_score(METRICS['auc']['y_true'],
                                                          METRICS['auc']['y_pred'])
            print(f'\nFinish EPOCH {epoch} with average_loss: {avg_loss: .5f} '
                  f'ROC_AUC_Score: {roc_auc_score:.5f}')
            
    # Clear the memory
    def clear_memory(self):
        del self.model, self.tokenizer
        libc.malloc_trim(0)
        gc.collect()

In [18]:
as_completed# Start training the model
display(train_data.head(3))

fold = 0
epochs = 1
trainer = TrainModelTPU(fold, epochs, train_data)

# Train the model
trainer.train_model()
# Validate the model
trainer.valid_model()

# Save model and tokenizer
trainer.save_model()
 
# Clear the memorys
trainer.clear_memory()
del trainer

,text,label,source,fold,aux_label
0,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,2.0,0
1,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,2.0,0
2,"Check whether the following essay is human written or AI generated? Remember, the text might ha...",0,train_essay,4.0,0


Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.35s/it]
/usr/local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
trainable params: 6,422,528 || all params: 8,544,103,424 || trainable%: 0.07516912754074827
Complete loading pretrained LLM model
Number_DEVICES: 8
BATCH_SIZE: 16, N_SAMPLES: 50162, STEPS_PER_EPOCH: 3135
TEXTS shape: (50162,)
AUX_LABELS shape: (50162, 1)
GENERATED shape: (50162, 1)
Complete creating optimizer and lr scheduler
optimizer AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.0003
    lr: 0.0
    maximize: False
    weight_decay: 2.9999999999999997e-05
)
lr_scheduler <torch.optim.lr_scheduler.LambdaLR object at 0x7e1f007b5db0>


  0%|          | 0/1 [00:00<?, ?it/s]

 01/01 | 3134/3135 lr: 7.83E-11, µ_loss: 0.560, step_loss: 0.362, µ_auc: 0.999

...

Start validating the model with number of sample 12541


100%|██████████| 783/783 [30:12<00:00,  2.31s/it]


Number of validation data 12541
loss (valid):  0.007
auc (valid): 1.000


/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/7b/2 - will assume that the vocabulary was not modified.
  warnings.warn(


Save the model and tokenizers to /kaggle/working/gemma7b_fold0_TPU


In [19]:
# from prettytable import PrettyTable

# def count_parameters(model):
#     table = PrettyTable(["Modules", "trainable", "Parameters"])
#     total_params = 0
#     trainable_params = 0
#     for name, parameter in model.named_parameters():
#         trainable = True
#         if not parameter.requires_grad:
#             trainable = False
#         params = parameter.numel()
#         table.add_row([name, trainable, params])
#         total_params += params
#         if trainable:
#             trainable_params += params
#     print(table)
#     print(f"Total Params: {total_params}")
#     print(f"Total Trainable Params: {trainable_params}, {trainable_params / total_params}% trainable")
#     return total_params
    
# count_parameters(trainer.model)

In [20]:
# # Start training the model
# display(train_data.head(3))
# try:
#     fold = 0
#     # train on fold-th data and epoch-th
#     for epoch in list(range(1, 11)):
#         trainer = TrainModelTPU(fold, epoch, train_data)
#         trainer.train_model() # Train the model    
#         trainer.valid_model() # Validate the model
#         trainer.save_model() # Save model and tokenizer
#         trainer.clear_memory()
#         del trainer
# except:
#     print("Something is wrong during training")
#     sys.exit(-1) # Stop with error
# print(f"Complete training the model on fold {fold}")
# sys.exit(0) # Stop the notebook normally


In [21]:
#     from __future__ import annotations
#     import time, sys, gc, logging, random
#     from pathlib import Path
#     import numpy as np
#     import pandas as pd
#     from tqdm import tqdm
#     from datasets import Dataset
#     from sklearn.model_selection import StratifiedKFold
#     from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig 
#     from peft import TaskType, prepare_model_for_kbit_training, AutoPeftModelForCausalLM # type: ignore
#     from transformers import BitsAndBytesConfig
#     import torch

#     from transformers import logging as hf_logging
#     from transformers import (
#         AutoTokenizer, AutoModelForSequenceClassification, LlamaForSequenceClassification,
#         TrainingArguments, Trainer, DataCollatorWithPadding)
#     import transformers

#     import peft
#     from accelerate import Accelerator
#     import bitsandbytes
#     from sklearn.metrics import accuracy_score, roc_auc_score
#     from shutil import rmtree
#     import language_tool_python
#     import optuna
#     import concurrent
#     from concurrent.futures import ThreadPoolExecutor
#     from concurrent.futures import wait

#     print(transformers.__version__)
#     print(peft.__version__)
#     print(torch.__version__)

#     

In [22]:
# class TrainModelGPU:
#     def __init__(self, fold):
#         self.fold = fold
#         self.TARGET_MODEL = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
#         self.load_model()

#     # Load the pretrained model and add an extra layer with PEFT library for fine-tuning
#     def load_model(self):
#         # Enable GPU to run the model with 4bit
#         bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
#         # LoRA: Low-Rank Adaptation of Large Language Models is a popular approach to fine-tune LLM on a single GPU
#         # https://abvijaykumar.medium.com/fine-tuning-llm-parameter-efficient-fine-tuning-peft-lora-qlora-part-2-d8e23877ac6f
#         lora_config = LoraConfig(            
#             r=16, # A larger 'r' value needs to update more parameters 
#             lora_dropout=0.10,
#             bias='none',
#             task_type=TaskType.SEQ_CLS,
#             inference_mode=False,
#             #Only targeting attention blocks of the model or targeting all linear layers
#             target_modules=["q_proj", "v_proj"] # if DEBUG else ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head'],                
#         )
#         # Load the tokenizer
#         self.tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
#         self.tokenizer.pad_token = self.tokenizer.eos_token

#         base_model = LlamaForSequenceClassification.from_pretrained(TARGET_MODEL,
#                                                                     num_labels=2, # label is 0 or 1
#                                                                     quantization_config=bnb_config,                                                                 
#                                                                     device_map="auto")

#         base_model.config.pretraining_tp = 1 # 1 is 7b
#         base_model.config.pad_token_id = tokenizer.pad_token_id

        
#         # Load the model
#         start = time.time()    
#         # Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of 
#         # pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. 
#         # training_model = prepare_model_for_kbit_training(base_model)
#         model = get_peft_model(base_model, lora_config)
#         print(f"Complete loading the base model {time.time() - start: .1f} seconds") 
#         model.print_trainable_parameters() # Display the trainable parameters
        
#         return model, tokenizer
    
#     def preprocess_function(self, examples, tokenizer, max_length=512):
#         examples["text"] = list(map(lambda text: self.pre_processing_text(text), examples["text"]))
#         return tokenizer(examples["text"], truncation=True, max_length=max_length, padding=True)

#     def compute_metrics(self, eval_pred):
#         predictions, labels = eval_pred
#         predictions = np.argmax(predictions, axis=1)

#         accuracy_val = accuracy_score(labels, predictions)
#         roc_auc_val = roc_auc_score(labels, predictions)
#         r = { "accuracy": accuracy_val,
#               "roc_auc": roc_auc_val}
#         # logging.debug(f'{r}')
#         return r

#     # Fine Tuning LLM: Parameter Efficient Fine Tuning (PEFT) and Lora configurations
#     # https://github.com/microsoft/LoRA
#     def train_model_by_fold(self, fold):
#         torch.cuda.empty_cache()
#         gc.collect()
#         print(f"Start training the fold {fold} model")
#         # Create train and valid dataset for a fold
#         fold_valid_df = train_data[train_data["fold"] == fold]
#         fold_train_df = train_data[train_data["fold"] != fold]
#         # Train the model with small (for debugging) or large samples
#         if DEBUG:
#             fold_train_df = fold_train_df.sample(frac =.1, random_state=SEED)
#             fold_valid_df = fold_valid_df.sample(frac =.3, random_state=SEED)
#         else:
#             fold_train_df = fold_train_df.sample(frac =.3, random_state=SEED)
#             fold_valid_df = fold_valid_df.sample(frac =1.0, random_state=SEED)

#         print(f'fold_train_df: Len = {len(fold_train_df)} Counts = {fold_train_df.groupby("fold")["label"].value_counts()}')
#         print(f'fold_valid_df: Len = {len(fold_valid_df)} Counts = {fold_valid_df.groupby("fold")["label"].value_counts()}')
#         # create the dataset
#         train_ds = Dataset.from_pandas(fold_train_df)
#         valid_ds = Dataset.from_pandas(fold_valid_df)

#         # Load the pretrained model and tokenizer
#         model, tokenizer = load_model(fold)

#         # Tokenize the train and valid dataset and pass tokenizer as function argument
#         train_tokenized_ds = train_ds.map(self.preprocess_function, batched=True,
#                                           fn_kwargs={"tokenizer": tokenizer})
#         valid_tokenized_ds = valid_ds.map(self.preprocess_function, batched=True,
#                                           fn_kwargs={"tokenizer": tokenizer})
#         # Create data collator with padding (padding to the longest sequence)
#         data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

#         # Start training processing        
#         TMP_DIR = Path(f"/kaggle/tmp/mistral_7b_fold{fold}/")
#         TMP_DIR.mkdir(exist_ok=True, parents=True)

#         STEPS = 5 if DEBUG else 20
#         EPOCHS = 1 if DEBUG else 10
#         LEARNING_RATE = 1e-4 
#         training_args = TrainingArguments(output_dir=TMP_DIR,
#                                           overwrite_output_dir=True,
#                                           fp16=True, #converts to float precision 16 using bitsandbytes
#                                           learning_rate=LEARNING_RATE,                                      
#                                           per_device_train_batch_size=1,
#                                           per_device_eval_batch_size=1,
#                                           gradient_accumulation_steps=16,
#                                           max_grad_norm=0.3,
#                                           optim='paged_adamw_32bit',
#                                           lr_scheduler_type="cosine",
#                                           num_train_epochs=EPOCHS,
#                                           weight_decay=0.01,
#                                           evaluation_strategy="epoch",
#                                           save_strategy="epoch",
#                                           load_best_model_at_end=True,
#                                           metric_for_best_model="roc_auc",
#                                           label_names=["label"],
#                                           push_to_hub=False,
#                                           warmup_steps=STEPS,
#                                           eval_steps=STEPS,
#                                           logging_steps=STEPS,
#                                           report_to='none', # if DEBUG else 'wandb'
#                                           log_level='warning', # 'warning' is default level 
#                                          )
#         start = time.time()
#         # Create the trainer 
#         trainer = Trainer(model=model,
#                           args=training_args,
#                           train_dataset=train_tokenized_ds,
#                           eval_dataset=valid_tokenized_ds,
#                           tokenizer=tokenizer,
#                           data_collator=data_collator,
#                           compute_metrics=self.compute_metrics)

#         trainer.train()

#         OUTPUT_DIR = Path(f"/kaggle/working/mistral_7b_fold{fold}/")
#         OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
#         # Save the full model and the training arguments
#         trainer.save_model(str(OUTPUT_DIR))
#         print(f"=== Finish the training for fold {fold} in {time.time() - start:.1f} seconds ===")
#         del model, trainer, tokenizer
#         torch.cuda.empty_cache()
#         gc.collect()